### Задача lookalike (Positive Unlabeled Learning)

Используем датасет mushroom и будем предсказывать ядовитые грибы

In [327]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split
#from sklearn.feature_extraction.text import TfidfVectorizer
import itertools

import matplotlib.pyplot as plt

%matplotlib inline
from catboost import CatBoostClassifier

In [328]:
df = pd.read_csv("mushroom/agaricus-lepiota.data")

In [329]:
df.head()

,classes,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


Так как все признаки датасета категориальные, воспользуемся catboost, с ним обработка информмации будет удобнее

In [330]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   classes                   8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [331]:
df.loc[df['classes']== 'p', 'classes'] = 1
df.loc[df['classes']== 'e', 'classes'] = 0
df['classes'] = df['classes'].astype(int)
df['classes'].value_counts()

0    4208
1    3916
Name: classes, dtype: int64

Дисбаланса классов нет

In [332]:
X = df.drop('classes', axis = 1)
y = df['classes']
cat_features = ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat']

In [333]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [334]:
X_train

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
5832,f,y,y,f,f,f,c,b,p,e,...,k,b,n,p,w,o,l,h,y,g
601,x,y,n,t,l,f,c,b,w,e,...,y,w,w,p,w,o,p,n,y,g
1601,f,s,g,f,n,f,w,b,k,t,...,f,w,w,p,w,o,e,k,a,g
4941,x,f,g,f,f,f,c,b,g,e,...,k,b,p,p,w,o,l,h,v,p
7492,f,y,n,f,f,f,c,n,b,t,...,k,p,w,p,w,o,e,w,v,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4931,x,y,e,t,n,f,c,b,e,e,...,s,e,e,p,w,t,e,w,c,w
3264,x,f,g,f,f,f,c,b,h,e,...,k,p,n,p,w,o,l,h,y,p
1653,x,s,g,f,n,f,w,b,h,t,...,s,w,w,p,w,o,e,n,s,g
2607,f,f,n,t,n,f,c,b,n,t,...,s,g,g,p,w,o,p,n,v,d


In [335]:
y_train

5832    1
601     0
1601    0
4941    1
7492    1
       ..
4931    0
3264    1
1653    0
2607    0
2732    0
Name: classes, Length: 6093, dtype: int64

In [336]:
classifier = CatBoostClassifier(random_state = 42, cat_features = cat_features)

classifier.fit(X_train, y_train)
preds = classifier.predict_proba(X_test)[:, 1]

Learning rate set to 0.022287
0:	learn: 0.6410732	total: 2.65ms	remaining: 2.65s
1:	learn: 0.5920969	total: 5.44ms	remaining: 2.71s
2:	learn: 0.5418579	total: 8.06ms	remaining: 2.68s
3:	learn: 0.4947208	total: 13.7ms	remaining: 3.42s
4:	learn: 0.4479437	total: 18.7ms	remaining: 3.73s
5:	learn: 0.4128656	total: 22ms	remaining: 3.64s
6:	learn: 0.3807262	total: 25.2ms	remaining: 3.58s
7:	learn: 0.3516865	total: 28.2ms	remaining: 3.49s
8:	learn: 0.3270496	total: 31.7ms	remaining: 3.49s
9:	learn: 0.3043793	total: 35.1ms	remaining: 3.47s
10:	learn: 0.2777046	total: 39.6ms	remaining: 3.56s
11:	learn: 0.2589411	total: 43.2ms	remaining: 3.56s
12:	learn: 0.2423441	total: 45.9ms	remaining: 3.49s
13:	learn: 0.2275524	total: 49.6ms	remaining: 3.49s
14:	learn: 0.2083961	total: 54.8ms	remaining: 3.6s
15:	learn: 0.1864441	total: 59.9ms	remaining: 3.68s
16:	learn: 0.1669339	total: 65.2ms	remaining: 3.77s
17:	learn: 0.1499582	total: 69.9ms	remaining: 3.81s
18:	learn: 0.1386028	total: 74.3ms	remaining: 3

178:	learn: 0.0027122	total: 994ms	remaining: 4.56s
179:	learn: 0.0026925	total: 1s	remaining: 4.56s
180:	learn: 0.0026717	total: 1.01s	remaining: 4.56s
181:	learn: 0.0026717	total: 1.01s	remaining: 4.56s
182:	learn: 0.0026598	total: 1.02s	remaining: 4.56s
183:	learn: 0.0026477	total: 1.03s	remaining: 4.56s
184:	learn: 0.0026375	total: 1.03s	remaining: 4.55s
185:	learn: 0.0025740	total: 1.04s	remaining: 4.55s
186:	learn: 0.0025688	total: 1.05s	remaining: 4.55s
187:	learn: 0.0025688	total: 1.05s	remaining: 4.54s
188:	learn: 0.0025602	total: 1.06s	remaining: 4.54s
189:	learn: 0.0025396	total: 1.06s	remaining: 4.53s
190:	learn: 0.0025285	total: 1.07s	remaining: 4.53s
191:	learn: 0.0025285	total: 1.08s	remaining: 4.53s
192:	learn: 0.0025007	total: 1.08s	remaining: 4.53s
193:	learn: 0.0025007	total: 1.09s	remaining: 4.52s
194:	learn: 0.0024684	total: 1.09s	remaining: 4.52s
195:	learn: 0.0024346	total: 1.1s	remaining: 4.51s
196:	learn: 0.0024327	total: 1.11s	remaining: 4.51s
197:	learn: 0.00

352:	learn: 0.0011852	total: 2s	remaining: 3.66s
353:	learn: 0.0011852	total: 2s	remaining: 3.65s
354:	learn: 0.0011852	total: 2.01s	remaining: 3.65s
355:	learn: 0.0011852	total: 2.01s	remaining: 3.64s
356:	learn: 0.0011827	total: 2.02s	remaining: 3.64s
357:	learn: 0.0011827	total: 2.02s	remaining: 3.63s
358:	learn: 0.0011746	total: 2.03s	remaining: 3.63s
359:	learn: 0.0011688	total: 2.04s	remaining: 3.62s
360:	learn: 0.0011688	total: 2.04s	remaining: 3.61s
361:	learn: 0.0011596	total: 2.05s	remaining: 3.61s
362:	learn: 0.0011596	total: 2.05s	remaining: 3.6s
363:	learn: 0.0011596	total: 2.06s	remaining: 3.6s
364:	learn: 0.0011361	total: 2.06s	remaining: 3.59s
365:	learn: 0.0011361	total: 2.07s	remaining: 3.58s
366:	learn: 0.0011271	total: 2.07s	remaining: 3.58s
367:	learn: 0.0011208	total: 2.08s	remaining: 3.57s
368:	learn: 0.0011194	total: 2.09s	remaining: 3.57s
369:	learn: 0.0011193	total: 2.09s	remaining: 3.56s
370:	learn: 0.0011193	total: 2.1s	remaining: 3.56s
371:	learn: 0.0011193

536:	learn: 0.0008690	total: 3s	remaining: 2.58s
537:	learn: 0.0008690	total: 3s	remaining: 2.58s
538:	learn: 0.0008690	total: 3.01s	remaining: 2.57s
539:	learn: 0.0008690	total: 3.01s	remaining: 2.57s
540:	learn: 0.0008689	total: 3.02s	remaining: 2.56s
541:	learn: 0.0008689	total: 3.02s	remaining: 2.56s
542:	learn: 0.0008689	total: 3.03s	remaining: 2.55s
543:	learn: 0.0008689	total: 3.03s	remaining: 2.54s
544:	learn: 0.0008689	total: 3.04s	remaining: 2.54s
545:	learn: 0.0008689	total: 3.04s	remaining: 2.53s
546:	learn: 0.0008689	total: 3.05s	remaining: 2.53s
547:	learn: 0.0008689	total: 3.06s	remaining: 2.52s
548:	learn: 0.0008689	total: 3.06s	remaining: 2.51s
549:	learn: 0.0008689	total: 3.06s	remaining: 2.51s
550:	learn: 0.0008689	total: 3.07s	remaining: 2.5s
551:	learn: 0.0008607	total: 3.08s	remaining: 2.5s
552:	learn: 0.0008527	total: 3.08s	remaining: 2.49s
553:	learn: 0.0008527	total: 3.09s	remaining: 2.48s
554:	learn: 0.0008517	total: 3.09s	remaining: 2.48s
555:	learn: 0.000851

724:	learn: 0.0008324	total: 3.99s	remaining: 1.51s
725:	learn: 0.0008324	total: 4s	remaining: 1.51s
726:	learn: 0.0008324	total: 4s	remaining: 1.5s
727:	learn: 0.0008324	total: 4.01s	remaining: 1.5s
728:	learn: 0.0008324	total: 4.01s	remaining: 1.49s
729:	learn: 0.0008324	total: 4.02s	remaining: 1.49s
730:	learn: 0.0008324	total: 4.03s	remaining: 1.48s
731:	learn: 0.0008324	total: 4.03s	remaining: 1.48s
732:	learn: 0.0008324	total: 4.04s	remaining: 1.47s
733:	learn: 0.0008324	total: 4.04s	remaining: 1.47s
734:	learn: 0.0008324	total: 4.05s	remaining: 1.46s
735:	learn: 0.0008324	total: 4.05s	remaining: 1.45s
736:	learn: 0.0008324	total: 4.06s	remaining: 1.45s
737:	learn: 0.0008324	total: 4.06s	remaining: 1.44s
738:	learn: 0.0008324	total: 4.07s	remaining: 1.44s
739:	learn: 0.0008324	total: 4.07s	remaining: 1.43s
740:	learn: 0.0008324	total: 4.08s	remaining: 1.43s
741:	learn: 0.0008324	total: 4.08s	remaining: 1.42s
742:	learn: 0.0008324	total: 4.09s	remaining: 1.41s
743:	learn: 0.000832

913:	learn: 0.0008322	total: 4.99s	remaining: 470ms
914:	learn: 0.0008322	total: 5s	remaining: 464ms
915:	learn: 0.0008322	total: 5s	remaining: 459ms
916:	learn: 0.0008322	total: 5.01s	remaining: 453ms
917:	learn: 0.0008322	total: 5.01s	remaining: 448ms
918:	learn: 0.0008322	total: 5.02s	remaining: 442ms
919:	learn: 0.0008322	total: 5.02s	remaining: 437ms
920:	learn: 0.0008322	total: 5.03s	remaining: 431ms
921:	learn: 0.0008322	total: 5.03s	remaining: 426ms
922:	learn: 0.0008322	total: 5.04s	remaining: 420ms
923:	learn: 0.0008322	total: 5.04s	remaining: 415ms
924:	learn: 0.0008322	total: 5.05s	remaining: 409ms
925:	learn: 0.0008322	total: 5.05s	remaining: 404ms
926:	learn: 0.0008322	total: 5.06s	remaining: 398ms
927:	learn: 0.0008322	total: 5.07s	remaining: 393ms
928:	learn: 0.0008322	total: 5.07s	remaining: 388ms
929:	learn: 0.0008322	total: 5.08s	remaining: 382ms
930:	learn: 0.0008322	total: 5.08s	remaining: 377ms
931:	learn: 0.0008322	total: 5.09s	remaining: 371ms
932:	learn: 0.0008

In [337]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [339]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

roc = roc_auc_score(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.983740, F-Score=1.000, Precision=1.000, Recall=1.000


Вывод: данные в датасете очень хорошо разделимы, наша модель определяет тип гриба со 100% точностью. Воспользуемся данными для построения модели pu learning.

In [340]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    f1 = f1_score(y_test, y_predict)
    roc = roc_auc_score(y_test, y_predict)
    rec = recall_score(y_test, y_predict, average='binary')
    prc = precision_score(y_test, y_predict, average='binary')
    return [f1, roc, rec, prc]

In [346]:
evaluate_list = [['100', fscore[ix], roc, recall[ix], precision[ix]]]

for el in [0.1, 0.25, 0.5, 0.7]:
    print('Доля отмеченных позитивных: ', el)

    mod_data = df.copy()
    #get the indices of the positives samples
    pos_ind = np.where(mod_data.iloc[:,0].values == 1)[0]
    #shuffle them
    np.random.shuffle(pos_ind)
    # leave just el% of the positives marked
    pos_sample_len = int(np.ceil(el * len(pos_ind)))
    print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
    pos_sample = pos_ind[:pos_sample_len]

    mod_data['class_test'] = 0
    mod_data.loc[pos_sample,'class_test'] = 1
    print('target variable:\n',mod_data.iloc[:,-1].value_counts())

    mod_data = mod_data.sample(frac=1)
    neg_sample = mod_data[mod_data['class_test']==0][:len(mod_data[mod_data['class_test']==1])]
    sample_test = mod_data[mod_data['class_test']==0][len(mod_data[mod_data['class_test']==1]):]
    pos_sample = mod_data[mod_data['class_test']==1]
    print(neg_sample.shape, pos_sample.shape)
    sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

    model = CatBoostClassifier(random_state = 42, cat_features = cat_features)

    model.fit(sample_train.iloc[:,1:-1], 
              sample_train.iloc[:,-1])
    y_predict = model.predict(sample_test.iloc[:,1:-1])

    evaluate_list.append([el*100]+evaluate_results(sample_test.iloc[:,0], y_predict))

Доля отмеченных позитивных:  0.1
Using 392/3916 as positives and unlabeling the rest
target variable:
 0    7732
1     392
Name: class_test, dtype: int64
(392, 24) (392, 24)
Learning rate set to 0.009286
0:	learn: 0.6893349	total: 1.53ms	remaining: 1.53s
1:	learn: 0.6842083	total: 4.2ms	remaining: 2.09s
2:	learn: 0.6803502	total: 5.99ms	remaining: 1.99s
3:	learn: 0.6757354	total: 9.01ms	remaining: 2.24s
4:	learn: 0.6727070	total: 9.84ms	remaining: 1.96s
5:	learn: 0.6694585	total: 10.8ms	remaining: 1.79s
6:	learn: 0.6662808	total: 11.7ms	remaining: 1.66s
7:	learn: 0.6630285	total: 14.2ms	remaining: 1.76s
8:	learn: 0.6587062	total: 16.6ms	remaining: 1.83s
9:	learn: 0.6556019	total: 19.4ms	remaining: 1.92s
10:	learn: 0.6510610	total: 20.9ms	remaining: 1.88s
11:	learn: 0.6463728	total: 23.4ms	remaining: 1.93s
12:	learn: 0.6431128	total: 26.3ms	remaining: 2s
13:	learn: 0.6399089	total: 29ms	remaining: 2.04s
14:	learn: 0.6374105	total: 29.9ms	remaining: 1.96s
15:	learn: 0.6331103	total: 32.6

160:	learn: 0.4841525	total: 375ms	remaining: 1.96s
161:	learn: 0.4839744	total: 378ms	remaining: 1.96s
162:	learn: 0.4837662	total: 381ms	remaining: 1.96s
163:	learn: 0.4833274	total: 384ms	remaining: 1.96s
164:	learn: 0.4830518	total: 388ms	remaining: 1.96s
165:	learn: 0.4828129	total: 392ms	remaining: 1.97s
166:	learn: 0.4824626	total: 395ms	remaining: 1.97s
167:	learn: 0.4821135	total: 399ms	remaining: 1.97s
168:	learn: 0.4817282	total: 402ms	remaining: 1.98s
169:	learn: 0.4811593	total: 406ms	remaining: 1.98s
170:	learn: 0.4805957	total: 409ms	remaining: 1.98s
171:	learn: 0.4803301	total: 412ms	remaining: 1.98s
172:	learn: 0.4801267	total: 415ms	remaining: 1.98s
173:	learn: 0.4796664	total: 418ms	remaining: 1.98s
174:	learn: 0.4792594	total: 421ms	remaining: 1.98s
175:	learn: 0.4788585	total: 424ms	remaining: 1.98s
176:	learn: 0.4783390	total: 427ms	remaining: 1.98s
177:	learn: 0.4779966	total: 429ms	remaining: 1.98s
178:	learn: 0.4775786	total: 432ms	remaining: 1.98s
179:	learn: 

381:	learn: 0.4376319	total: 964ms	remaining: 1.56s
382:	learn: 0.4374525	total: 967ms	remaining: 1.56s
383:	learn: 0.4372954	total: 970ms	remaining: 1.56s
384:	learn: 0.4371300	total: 973ms	remaining: 1.55s
385:	learn: 0.4369822	total: 977ms	remaining: 1.55s
386:	learn: 0.4367977	total: 980ms	remaining: 1.55s
387:	learn: 0.4367057	total: 982ms	remaining: 1.55s
388:	learn: 0.4364116	total: 986ms	remaining: 1.55s
389:	learn: 0.4363447	total: 989ms	remaining: 1.55s
390:	learn: 0.4362725	total: 991ms	remaining: 1.54s
391:	learn: 0.4362706	total: 992ms	remaining: 1.54s
392:	learn: 0.4361625	total: 995ms	remaining: 1.54s
393:	learn: 0.4360750	total: 998ms	remaining: 1.53s
394:	learn: 0.4358494	total: 1s	remaining: 1.53s
395:	learn: 0.4358246	total: 1s	remaining: 1.53s
396:	learn: 0.4356011	total: 1s	remaining: 1.53s
397:	learn: 0.4353773	total: 1.01s	remaining: 1.52s
398:	learn: 0.4352166	total: 1.01s	remaining: 1.52s
399:	learn: 0.4351711	total: 1.01s	remaining: 1.52s
400:	learn: 0.4349725

598:	learn: 0.4050198	total: 1.55s	remaining: 1.04s
599:	learn: 0.4047670	total: 1.55s	remaining: 1.04s
600:	learn: 0.4045077	total: 1.56s	remaining: 1.03s
601:	learn: 0.4043749	total: 1.56s	remaining: 1.03s
602:	learn: 0.4043081	total: 1.56s	remaining: 1.03s
603:	learn: 0.4042150	total: 1.57s	remaining: 1.03s
604:	learn: 0.4041127	total: 1.57s	remaining: 1.02s
605:	learn: 0.4040812	total: 1.57s	remaining: 1.02s
606:	learn: 0.4039341	total: 1.58s	remaining: 1.02s
607:	learn: 0.4038012	total: 1.58s	remaining: 1.02s
608:	learn: 0.4036038	total: 1.58s	remaining: 1.02s
609:	learn: 0.4034415	total: 1.59s	remaining: 1.01s
610:	learn: 0.4032935	total: 1.59s	remaining: 1.01s
611:	learn: 0.4031437	total: 1.59s	remaining: 1.01s
612:	learn: 0.4029601	total: 1.59s	remaining: 1.01s
613:	learn: 0.4029451	total: 1.59s	remaining: 1s
614:	learn: 0.4028617	total: 1.6s	remaining: 1s
615:	learn: 0.4027957	total: 1.6s	remaining: 999ms
616:	learn: 0.4026087	total: 1.6s	remaining: 996ms
617:	learn: 0.4024474

806:	learn: 0.3730629	total: 2.14s	remaining: 512ms
807:	learn: 0.3729486	total: 2.14s	remaining: 509ms
808:	learn: 0.3726055	total: 2.15s	remaining: 507ms
809:	learn: 0.3725806	total: 2.15s	remaining: 504ms
810:	learn: 0.3724061	total: 2.15s	remaining: 502ms
811:	learn: 0.3722604	total: 2.16s	remaining: 499ms
812:	learn: 0.3722033	total: 2.16s	remaining: 497ms
813:	learn: 0.3719826	total: 2.16s	remaining: 494ms
814:	learn: 0.3715068	total: 2.17s	remaining: 492ms
815:	learn: 0.3711440	total: 2.17s	remaining: 489ms
816:	learn: 0.3711307	total: 2.17s	remaining: 487ms
817:	learn: 0.3709011	total: 2.17s	remaining: 484ms
818:	learn: 0.3706912	total: 2.18s	remaining: 481ms
819:	learn: 0.3704677	total: 2.18s	remaining: 479ms
820:	learn: 0.3703459	total: 2.18s	remaining: 476ms
821:	learn: 0.3702911	total: 2.19s	remaining: 473ms
822:	learn: 0.3702022	total: 2.19s	remaining: 471ms
823:	learn: 0.3699991	total: 2.19s	remaining: 468ms
824:	learn: 0.3698986	total: 2.19s	remaining: 466ms
825:	learn: 

...

265:	learn: 0.2846452	total: 1.37s	remaining: 3.79s
266:	learn: 0.2846452	total: 1.37s	remaining: 3.77s
267:	learn: 0.2845941	total: 1.38s	remaining: 3.77s
268:	learn: 0.2845645	total: 1.39s	remaining: 3.77s
269:	learn: 0.2844662	total: 1.39s	remaining: 3.77s
270:	learn: 0.2844492	total: 1.4s	remaining: 3.76s
271:	learn: 0.2843864	total: 1.4s	remaining: 3.76s
272:	learn: 0.2842511	total: 1.41s	remaining: 3.75s
273:	learn: 0.2842322	total: 1.42s	remaining: 3.75s
274:	learn: 0.2841971	total: 1.42s	remaining: 3.75s
275:	learn: 0.2840986	total: 1.43s	remaining: 3.74s
276:	learn: 0.2840509	total: 1.43s	remaining: 3.74s
277:	learn: 0.2839417	total: 1.44s	remaining: 3.74s
278:	learn: 0.2838323	total: 1.44s	remaining: 3.73s
279:	learn: 0.2837896	total: 1.45s	remaining: 3.73s
280:	learn: 0.2837000	total: 1.46s	remaining: 3.72s
281:	learn: 0.2836440	total: 1.46s	remaining: 3.72s
282:	learn: 0.2836191	total: 1.47s	remaining: 3.71s
283:	learn: 0.2835966	total: 1.47s	remaining: 3.71s
284:	learn: 0.

441:	learn: 0.2746101	total: 2.37s	remaining: 3s
442:	learn: 0.2745648	total: 2.38s	remaining: 2.99s
443:	learn: 0.2745311	total: 2.39s	remaining: 2.99s
444:	learn: 0.2745069	total: 2.39s	remaining: 2.99s
445:	learn: 0.2744298	total: 2.4s	remaining: 2.98s
446:	learn: 0.2744005	total: 2.41s	remaining: 2.98s
447:	learn: 0.2743830	total: 2.41s	remaining: 2.97s
448:	learn: 0.2743464	total: 2.42s	remaining: 2.97s
449:	learn: 0.2742413	total: 2.42s	remaining: 2.96s
450:	learn: 0.2741925	total: 2.43s	remaining: 2.96s
451:	learn: 0.2741408	total: 2.44s	remaining: 2.95s
452:	learn: 0.2740274	total: 2.44s	remaining: 2.95s
453:	learn: 0.2739183	total: 2.45s	remaining: 2.94s
454:	learn: 0.2738814	total: 2.45s	remaining: 2.94s
455:	learn: 0.2738230	total: 2.46s	remaining: 2.93s
456:	learn: 0.2737815	total: 2.46s	remaining: 2.93s
457:	learn: 0.2737349	total: 2.47s	remaining: 2.92s
458:	learn: 0.2736866	total: 2.48s	remaining: 2.92s
459:	learn: 0.2736297	total: 2.48s	remaining: 2.91s
460:	learn: 0.27

610:	learn: 0.2661795	total: 3.38s	remaining: 2.15s
611:	learn: 0.2661679	total: 3.38s	remaining: 2.15s
612:	learn: 0.2661126	total: 3.39s	remaining: 2.14s
613:	learn: 0.2660137	total: 3.4s	remaining: 2.14s
614:	learn: 0.2659402	total: 3.4s	remaining: 2.13s
615:	learn: 0.2658979	total: 3.41s	remaining: 2.13s
616:	learn: 0.2658884	total: 3.42s	remaining: 2.12s
617:	learn: 0.2658088	total: 3.42s	remaining: 2.11s
618:	learn: 0.2657823	total: 3.43s	remaining: 2.11s
619:	learn: 0.2656501	total: 3.43s	remaining: 2.1s
620:	learn: 0.2656395	total: 3.44s	remaining: 2.1s
621:	learn: 0.2655793	total: 3.44s	remaining: 2.09s
622:	learn: 0.2654448	total: 3.45s	remaining: 2.09s
623:	learn: 0.2654004	total: 3.46s	remaining: 2.08s
624:	learn: 0.2653772	total: 3.46s	remaining: 2.08s
625:	learn: 0.2653498	total: 3.47s	remaining: 2.07s
626:	learn: 0.2653412	total: 3.47s	remaining: 2.06s
627:	learn: 0.2653011	total: 3.48s	remaining: 2.06s
628:	learn: 0.2652606	total: 3.48s	remaining: 2.06s
629:	learn: 0.26

780:	learn: 0.2567611	total: 4.38s	remaining: 1.23s
781:	learn: 0.2567458	total: 4.39s	remaining: 1.22s
782:	learn: 0.2567356	total: 4.39s	remaining: 1.22s
783:	learn: 0.2566755	total: 4.4s	remaining: 1.21s
784:	learn: 0.2566395	total: 4.41s	remaining: 1.21s
785:	learn: 0.2565435	total: 4.41s	remaining: 1.2s
786:	learn: 0.2564959	total: 4.42s	remaining: 1.2s
787:	learn: 0.2564775	total: 4.42s	remaining: 1.19s
788:	learn: 0.2563593	total: 4.43s	remaining: 1.18s
789:	learn: 0.2562078	total: 4.44s	remaining: 1.18s
790:	learn: 0.2561708	total: 4.44s	remaining: 1.17s
791:	learn: 0.2560864	total: 4.45s	remaining: 1.17s
792:	learn: 0.2559524	total: 4.45s	remaining: 1.16s
793:	learn: 0.2559206	total: 4.46s	remaining: 1.16s
794:	learn: 0.2558863	total: 4.47s	remaining: 1.15s
795:	learn: 0.2558613	total: 4.47s	remaining: 1.15s
796:	learn: 0.2558407	total: 4.48s	remaining: 1.14s
797:	learn: 0.2558248	total: 4.48s	remaining: 1.13s
798:	learn: 0.2558072	total: 4.49s	remaining: 1.13s
799:	learn: 0.2

948:	learn: 0.2483049	total: 5.38s	remaining: 289ms
949:	learn: 0.2481663	total: 5.39s	remaining: 284ms
950:	learn: 0.2481540	total: 5.4s	remaining: 278ms
951:	learn: 0.2480268	total: 5.4s	remaining: 272ms
952:	learn: 0.2480200	total: 5.41s	remaining: 267ms
953:	learn: 0.2478885	total: 5.41s	remaining: 261ms
954:	learn: 0.2478474	total: 5.42s	remaining: 255ms
955:	learn: 0.2478356	total: 5.43s	remaining: 250ms
956:	learn: 0.2478272	total: 5.43s	remaining: 244ms
957:	learn: 0.2477590	total: 5.44s	remaining: 238ms
958:	learn: 0.2476909	total: 5.44s	remaining: 233ms
959:	learn: 0.2475839	total: 5.45s	remaining: 227ms
960:	learn: 0.2474655	total: 5.46s	remaining: 221ms
961:	learn: 0.2474188	total: 5.46s	remaining: 216ms
962:	learn: 0.2473551	total: 5.47s	remaining: 210ms
963:	learn: 0.2472823	total: 5.47s	remaining: 204ms
964:	learn: 0.2471980	total: 5.48s	remaining: 199ms
965:	learn: 0.2471588	total: 5.48s	remaining: 193ms
966:	learn: 0.2471474	total: 5.49s	remaining: 187ms
967:	learn: 0.

In [347]:
evaluate_table = pd.DataFrame(evaluate_list, columns = ['%', 'f1_score', 'roc_auc_score', 'recall', 'precision'])

In [348]:
evaluate_table

,%,f1_score,roc_auc_score,recall,precision
0,100,1.000000,1.000000,1.000000,1.0
1,10.0,0.961820,0.963224,0.926449,1.0
2,25.0,0.997816,0.997821,0.995642,1.0
3,50.0,0.999231,0.999231,0.998463,1.0
4,70.0,0.999106,0.999107,0.998214,1.0


Вывод: модель обладает отличной разделяющей способностью даже при обучении на 10% позитивного класса и unlabeled все остальные данные, так как данные очень хорошо разделимы. Ее показатели лишь слегка хуже обученной на истинных 100% значениях позитивного класса модели, у которой все показатели 1.0.